# 20190406 Selenium & BeautifulSoup 실습

### 인터파크 투어 Crawling

In [3]:
!pip install selenium

In [4]:
!pip install beautifulsoup4

In [16]:
#!pip install bs4
from selenium import webdriver as wd
from bs4 import BeautifulSoup as bs
import time
import pandas as pd
import requests

In [19]:
main_url = 'http://tour.interpark.com/'
keyword = '유럽'
path = 'C:/Users/yoonj/Downloads/chromedriver.exe'
tour_list = []

In [20]:
driver = wd.Chrome(path)
driver.get(main_url)

In [21]:
# 검색창에 keyword 입력하기
search_box = driver.find_element_by_css_selector('#SearchGNBText')
search_box.clear()
search_box.send_keys(keyword)

In [22]:
# 검색버튼 클릭하기
search_btn = driver.find_element_by_css_selector('.search-btn')
search_btn.click()

In [23]:
# 해외여행 더보기 클릭하기
driver.find_element_by_css_selector('body > div.container > div > div > div.panelZone > div.oTravelBox > ul > li.moreBtnWrap > button').click()

In [24]:
# 상품 리스트 만들기
boxItems = driver.find_elements_by_css_selector('.oTravelBox>.boxList>li')
type(boxItems), len(boxItems)

(list, 11)

In [25]:
# 상품의 제목, 가격, 출발가능기간, 랑크 뽑아내서 리스트 만들기
for li in boxItems:
    elem = [li.find_element_by_css_selector('h5.proTit').text,
            li.find_element_by_css_selector('.proPrice').text,
            li.find_elements_by_css_selector('.info-row .proInfo')[1].text,
            li.find_element_by_css_selector('a').get_attribute('onclick')]
    tour_list.append(elem)
pd.DataFrame(tour_list, columns=['title', 'price', 'date', 'href']).head()

,title,price,date,href
0,[맞춤 여행]서유럽 자유 여행 맞춤 견적 문의,"10,000 원~",출발 가능 기간 : 2019.04.06~2020.01.31,searchModule.OnClickDetail('http://tour.interp...
1,(직항) ★BEST인기상품★ 발칸/동유럽 4국9일,"1,590,000 원~",출발 가능 기간 : 2019.04.07~2019.10.31,searchModule.OnClickDetail('http://tour.interp...
2,[1일1팩] ♥동유럽의 봄이오면♪_TK,"999,000 원~",출발 가능 기간 : 2019.04.07~2019.10.31,searchModule.OnClickDetail('http://tour.interp...
3,[나만의 특별한 유럽♥]스페인+포르투갈 10일_대한항공/1급호텔UP/항공1회/포르투관광,"2,557,800 원~",출발 가능 기간 : 2019.04.07~2019.10.29,searchModule.OnClickDetail('http://tour.interp...
4,[A380★두바이]서유럽4국10일/리기산+로마 아울렛,"1,890,000 원~",출발 가능 기간 : 2019.04.09~2019.10.30,searchModule.OnClickDetail('http://tour.interp...


In [26]:
# href에서 적절한 url 뽑아낸 후 그 페이지에서 내용 추출하기
for tour in tour_list:
    href = tour[3].split(',')[0].replace('searchModule.OnClickDetail(','')[1:-1]  # href에서 url 주소 뽑아내기
    respond = requests.get(href)  # HTTP get request
    html = respond.text  # HTML 소스 가져오기
    soup = bs(html, 'html.parser')  # HTML 소스를 python 객체로 변환하기
    data = soup.find('div', 'tip-cover')  # '인터파크 혜택' 뽑아내기
    tour.append(data.text)
pd.DataFrame(tour_list, columns=['title', 'price', 'date', 'href', 'content']).head()

,title,price,date,href,content
0,[맞춤 여행]서유럽 자유 여행 맞춤 견적 문의,"10,000 원~",출발 가능 기간 : 2019.04.06~2020.01.31,searchModule.OnClickDetail('http://tour.interp...,\n\n\n인터파크 혜택\n 인터파크투어 유럽 자유여행 견적 문의~!고객님의 여행...
1,(직항) ★BEST인기상품★ 발칸/동유럽 4국9일,"1,590,000 원~",출발 가능 기간 : 2019.04.07~2019.10.31,searchModule.OnClickDetail('http://tour.interp...,"\n\n\n인터파크 혜택\n 최고의 만족도, 인터파크투어 BEST상품♥ ★★ 왜 ..."
2,[1일1팩] ♥동유럽의 봄이오면♪_TK,"999,000 원~",출발 가능 기간 : 2019.04.07~2019.10.31,searchModule.OnClickDetail('http://tour.interp...,\n\n\n인터파크 혜택\n 동유럽 일주 4국9일 체코 ♥ 오스트리아 ♥ 헝가리 ...
3,[나만의 특별한 유럽♥]스페인+포르투갈 10일_대한항공/1급호텔UP/항공1회/포르투관광,"2,557,800 원~",출발 가능 기간 : 2019.04.07~2019.10.29,searchModule.OnClickDetail('http://tour.interp...,\n\n\n인터파크 혜택\n 나만의 특별한 유럽♥ 스페인/포르투갈10일＃포르투갈 포...
4,[A380★두바이]서유럽4국10일/리기산+로마 아울렛,"1,890,000 원~",출발 가능 기간 : 2019.04.09~2019.10.30,searchModule.OnClickDetail('http://tour.interp...,\n\n\n인터파크 혜택\n 2017년 트...


In [27]:
tour_list[0][4]

'\n\n\n인터파크 혜택\n\xa0 인터파크투어\xa0유럽 자유여행 견적 문의~!고객님의 여행 플래너가 되어 유럽여행을 좀더 쉽고\xa0알차게 다녀오실 수 있는 일정으로 만들어 드립니다. 마음에 드는 일정이 없다고 걱정하지 마세요. 맞춤견적이라고 절대 비싸지 않습니다.원하시는 일정에 최대한의 저렴하고 편리한 항공일정을 찾아서 견적해 드립니다. \xa0\xa0\xa0★ 예약 방법 ★요청사항에 원하시는 여행일정을 적어주시면 담당자가 확인 후 전화 & 맞춤견적을 보내드립니다.※\xa0여행하시는 모든 분들의 여권과 동일한 영문명 / 성별 / 생년월일이 있어야 견적안내 가능합니다※ 관광지 입장권 / 식당추천 및 예약\xa0등은\xa0대행불가합니다\xa0'

In [28]:
driver.quit()

### Make a Request

In [29]:
# Begin by importing the Requests module
import requests

In [30]:
# Get a webpage
r = requests.get('https://www.naver.com')  # 'get' returns a resopnse object

In [31]:
# Read the content of the server's response
html = r.text # returns HTML source code
html

'<!doctype html>\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n<html lang="ko">\n<head>\n<meta charset="utf-8">\n<meta name="Referrer" content="origin">\n<meta http-equiv="Content-Script-Type" content="text/javascript">\n<meta http-equiv="Content-Style-Type" content="text/css">\n<meta http-equiv="X-UA-Compatible" content="IE=edge">\n<meta name="viewport" content="width=1100">\n<meta name="apple-mobile-web-app-title" content="NAVER" />\n<meta name="robots" content="index,nofollow"/>\n<meta name="description" content="네이버 메인에서 다양한 정보와 유용한 컨텐츠를 만나 보세요"/>\n<meta property="og:title" content="네이버">\n<meta property="og:url" content="https://www.naver.com/">\n<meta property="og:image" content="https://s.pstatic.net/static/www/mobile/edit/2016/0705/mobile_212852414260.png">\n<meta property="og:description" content="네이버 메인에서 다양한 정보와 유용한 컨텐츠를 만나 보세요"/>\n<meta name="twitter:card" content="summary">\n<meta name="twitter:title" content="">\n<meta name="twitter:url" content="https://www.naver.com/">\n<meta name="

### BeautifulSoup

In [32]:
from bs4 import BeautifulSoup as bs

In [33]:
# HTML 소스를 python 객체로 변환하기
# BeautifulSoup(html_source, parser)
soup = bs(html, 'html.parser')
soup

<!DOCTYPE doctype html>

<html lang="ko">
<head>
<meta charset="utf-8"/>
<meta content="origin" name="Referrer"/>
<meta content="text/javascript" http-equiv="Content-Script-Type"/>
<meta content="text/css" http-equiv="Content-Style-Type"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=1100" name="viewport"/>
<meta content="NAVER" name="apple-mobile-web-app-title">
<meta content="index,nofollow" name="robots">
<meta content="네이버 메인에서 다양한 정보와 유용한 컨텐츠를 만나 보세요" name="description">
<meta content="네이버" property="og:title"/>
<meta content="https://www.naver.com/" property="og:url"/>
<meta content="https://s.pstatic.net/static/www/mobile/edit/2016/0705/mobile_212852414260.png" property="og:image"/>
<meta content="네이버 메인에서 다양한 정보와 유용한 컨텐츠를 만나 보세요" property="og:description">
<meta content="summary" name="twitter:card"/>
<meta content="" name="twitter:title"/>
<meta content="https://www.naver.com/" name="twitter:url"/>
<meta content="https://s.pstatic.net/static/www/

In [34]:
# 특정 부분 추출하기
# soup.find(name=None, attrs={}, recursive=True, text=None, **kwargs)
soup.find('title')  # tag name이 'title'인 부분을 찾아준다

<title>NAVER</title>

In [35]:
soup.find('title').text  # string

'NAVER'

In [36]:
# 똑같은 tag name이 여러 개 존재할 경우
# soup.find_all(name=None, attrs={}, recursive=True, text=None, limit=None, **kwargs)
soup.find('a')  # 똑같은 tag name이 여러 개인 경우 find는 가장 먼저 나오는 하나만 return
soup.find_all('a')  # list type

[<a href="#news_cast" onclick="document.getElementById('news_cast2').tabIndex = -1;document.getElementById('news_cast2').focus();return false;"><span>연합뉴스 바로가기</span></a>,
 <a href="#themecast" onclick="document.getElementById('themecast').tabIndex = -1;document.getElementById('themecast').focus();return false;"><span>주제별캐스트 바로가기</span></a>,
 <a href="#time_square" onclick="document.getElementById('time_square').tabIndex = -1;document.getElementById('time_square').focus();return false;"><span>타임스퀘어 바로가기</span></a>,
 <a href="#shp_cst" onclick="document.getElementById('shp_cst').tabIndex = -1;document.getElementById('shp_cst').focus();return false;"><span>쇼핑캐스트 바로가기</span></a>,
 <a href="#account" onclick="document.getElementById('account').tabIndex = -1;document.getElementById('account').focus();return false;"><span>로그인 바로가기</span></a>,
 <a class="al_favorite" data-clk="top.mkhome" href="http://help.naver.com/support/alias/contents2/naverhome/naverhome_1.naver">네이버를 시작페이지로<span class="

In [37]:
# 실시간 검색어
ss = soup.find('div', 'ah_list').find_all('a', 'ah_a')
for i in ss:
    print(i.find('span', 'ah_r').text, i.find('span', 'ah_k').text)

1 원더풀 고스트
2 속초시장
3 맛있는 녀석들 동태찌개
4 노유정
5 아름다운 세상
6 let on
7 기안84
8 discern
9 공무원시험
10 드라이브
11 stand out
12 2019 국가직 공무원 시험일정
13 너 사용법
14 노엘
15 왁스
16 중앙소방학교
17 고등래퍼3 우승자
18 김민규
19 이영범
20 김철수 속초시장


In [40]:
from six.moves import urllib

In [44]:
# 먼저 BeautifulSoup을 생성하고 인자로 네이버 영화 랭킹 페이지 전달
soup2 = bs(urllib.request.urlopen('http://movie.naver.com/movie/sdb/rank/rmovie.nhn?sel=pnt&date=20161120').read(), 'html.parser')
# 해당 페이지가 어떤 tag들로 구성되어있는지 보고싶을 때
print(soup2.prettify())
# 아니면 개발자 도구 봐도 됨

<!DOCTYPE html>
<html lang="ko">
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <meta content="http://imgmovie.naver.com/today/naverme/naverme_profile.jpg" property="me2:image">
   <meta content="네이버영화 " property="me2:post_tag">
    <meta content="네이버영화" property="me2:category1"/>
    <meta content="" property="me2:category2"/>
    <meta content="랭킹 : 네이버 영화" property="og:title"/>
    <meta content="영화, 영화인, 예매, 박스오피스 랭킹 정보 제공" property="og:description"/>
    <meta content="article" property="og:type"/>
    <meta content="https://movie.naver.com/movie/sdb/rank/rmovie.nhn?sel=pnt&amp;date=20161120" property="og:url"/>
    <meta content="http://static.naver.net/m/movie/icons/OG_270_270.png" property="og:image"/>
    <!-- http://static.naver.net/m/movie/im/navermovie.jpg -->
    <meta content="http://imgmovie.naver.com/today/naverme/naverme_profile.jpg" property="og:article:thumbnailUrl"/>
    <meta

In [45]:
# <div class='tit5'
res = soup2.find_all('div', 'tit5')
print(res)

[<div class="tit5">
<a href="/movie/bi/mi/basic.nhn?code=17421" title="쇼생크 탈출">쇼생크 탈출</a>
</div>, <div class="tit5">
<a href="/movie/bi/mi/basic.nhn?code=10200" title="터미네이터 2">터미네이터 2</a>
</div>, <div class="tit5">
<a href="/movie/bi/mi/basic.nhn?code=146534" title="자백">자백</a>
</div>, <div class="tit5">
<a href="/movie/bi/mi/basic.nhn?code=10048" title="죽은 시인의 사회">죽은 시인의 사회</a>
</div>, <div class="tit5">
<a href="/movie/bi/mi/basic.nhn?code=22126" title="인생은 아름다워">인생은 아름다워</a>
</div>, <div class="tit5">
<a href="/movie/bi/mi/basic.nhn?code=17170" title="레옹">레옹</a>
</div>, <div class="tit5">
<a href="/movie/bi/mi/basic.nhn?code=24452" title="매트릭스">매트릭스</a>
</div>, <div class="tit5">
<a href="/movie/bi/mi/basic.nhn?code=10002" title="빽 투 더 퓨쳐">빽 투 더 퓨쳐</a>
</div>, <div class="tit5">
<a href="/movie/bi/mi/basic.nhn?code=32686" title="센과 치히로의 행방불명">센과 치히로의 행방불명</a>
</div>, <div class="tit5">
<a href="/movie/bi/mi/basic.nhn?code=69105" title="월-E">월-E</a>
</div>, <div class="tit5">
<a href

In [46]:
for n in res:
    # get_text()를 이용하면 해당 tag 에서 text정보만 가져올 수 있음
    print(n.get_text())


쇼생크 탈출


터미네이터 2


자백


죽은 시인의 사회


인생은 아름다워


레옹


매트릭스


빽 투 더 퓨쳐


센과 치히로의 행방불명


월-E


살인의 추억


포레스트 검프


반지의 제왕: 왕의 귀환


토이 스토리 3


나 홀로 집에


사운드 오브 뮤직


라이언 일병 구하기


굿바이 마이 프랜드


에이리언 2


주토피아


여인의 향기


헬프


동주


글래디에이터


패왕별희


미세스 다웃파이어


울지마 톤즈


반지의 제왕: 두 개의 탑


아마데우스


세 얼간이


클래식


그대를 사랑합니다


언터처블: 1%의 우정


드래곤 길들이기


인어 공주


오페라의 유령 : 25주년 특별 공연


서유기 2 - 선리기연


모노노케 히메


빌리 엘리어트 뮤지컬 라이브


라푼젤


바이센테니얼 맨


집으로...


아이언 자이언트


캐스트 어웨이


굿 윌 헌팅


쉰들러 리스트


프리퀀시


소원


지금, 만나러 갑니다


다크 나이트

